# Project Objectives:

Provider Fraud is one of the biggest problems facing Medicare. According to the government, the total Medicare spending increased exponentially due to frauds in Medicare claims. Healthcare fraud is an organized crime which involves peers of providers, physicians, beneficiaries acting together to make fraud claims.

Rigorous analysis of Medicare data has yielded many physicians who indulge in fraud. They adopt ways in which an ambiguous diagnosis code is used to adopt costliest procedures and drugs. Insurance companies are the most vulnerable institutions impacted due to these bad practices. Due to this reason, insurance companies increased their insurance premiums and as result healthcare is becoming costly matter day by day.

Healthcare fraud and abuse take many forms. Some of the most common types of frauds by providers are:

a) Billing for services that were not provided.

b) Duplicate submission of a claim for the same service.

c) Misrepresenting the service provided.

d) Charging for a more complex or expensive service than was actually provided.

e) Billing for a covered service when the service actually provided was not covered.

# Problem Statement:

The goal of this project is to " predict the potentially fraudulent providers " based on the claims filed by them.along with this, we will also discover important variables helpful in detecting the behaviour of potentially fraud providers. further, we will study fraudulent patterns in the provider's claims to understand the future behaviour of providers.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot 
from numpy import sort
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")

# Introduction to the Dataset:

For the purpose of this project, we are considering Inpatient claims, Outpatient claims and Beneficiary details of each provider. Lets see their details :

A) Inpatient Data

This data provides insights about the claims filed for those patients who are admitted in the hospitals. It also provides additional details like their admission and discharge dates and admit d diagnosis code.

B) Outpatient Data

This data provides details about the claims filed for those patients who visit hospitals and not admitted in it.

C) Beneficiary Details Data

This data contains beneficiary KYC details like health conditions,region they belong to etc.

D) Provider Data

This data contains whether a Provider fraud or not

- The biggest challange for these datasets is all these datas are not in Provider level. Beneficiary Data is at beneficiary level and Provider data is at provider level and IN & OUT patient datas are at claim id level. But we have to make the Whole data at Provider level. So our real work starts here.

- In the data of Inpatients and Outpatients there are many providers. Under each provider there are at least one beneficiary. Each beneficiary has at least one claim id. in the data claim ids are unique.Our real job is to aggregate the datas against each Providers and to analyse them. So the first work of us is to make the datas from each datasets 

In [4]:
# to load the datasets
train_provider=pd.read_csv("C:\\Users\\Sahil\\Desktop\\Summer\\Project\\Dataset\\Train-1542865627584.csv")
train_ben=pd.read_csv("C:\\Users\\Sahil\\Desktop\\Summer\\Project\\Dataset\\Train_Beneficiarydata-1542865627584.csv")
train_in=pd.read_csv("C:\\Users\\Sahil\\Desktop\\Summer\\Project\\Dataset\\Train_Inpatientdata-1542865627584.csv")
train_out=pd.read_csv("C:\\Users\\Sahil\\Desktop\\Summer\\Project\\Dataset\\Train_Outpatientdata-1542865627584.csv")

In the Provider dataset our target variable is PotentialFraud and the two values of target variable is 'Yes' and 'No'. Now we convert them into 1 and 0 respectively

In [3]:


train_provider=train_provider.replace({'PotentialFraud':'Yes'},1)
train_provider=train_provider.replace({'PotentialFraud':'No'},0)

In [4]:
train_provider.shape

(5410, 2)

In [5]:
train_ben.shape

(138556, 25)

In [6]:
train_in.shape

(40474, 30)

In [7]:
train_out.shape

(517737, 27)

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
train_in.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [7]:
train_ben.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200


Lets Create Age column to the  Beneficiary dataset (train_ben) by deducting date of birth from date of death. as the dates were in string we have to convert them in date format (y-m-d). then we create age column

In [8]:


train_ben['DOB'] = pd.to_datetime(train_ben['DOB'] , format = '%Y-%m-%d')
train_ben['DOD'] = pd.to_datetime(train_ben['DOD'],format = '%Y-%m-%d',errors='ignore')
train_ben['Age'] = round(((train_ben['DOD'] - train_ben['DOB']).dt.days)/365)

In [9]:
train_ben.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age
0,BENE11001,1943-01-01,NaT,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,NaN
1,BENE11002,1936-09-01,NaT,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50,NaN
2,BENE11003,1936-08-01,NaT,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40,NaN
3,BENE11004,1922-07-01,NaT,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,NaN
4,BENE11005,1935-09-01,NaT,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200,NaN


for some of the beneficiaries the DOD is Null. Now we have to impute age for those beneficiaries
As we see that last DOD value is 2009-12-01 ,which means Beneficiary Details data is of year 2009.
so we will calculate age of other benficiaries for year 2009.

In [10]:


train_ben.Age.fillna(round(((pd.to_datetime('2009-12-01' , format = '%Y-%m-%d') - train_ben['DOB']).dt.days)/365),
                                 inplace=True)

In [11]:
train_ben.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age
0,BENE11001,1943-01-01,NaT,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,67.0
1,BENE11002,1936-09-01,NaT,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50,73.0
2,BENE11003,1936-08-01,NaT,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40,73.0
3,BENE11004,1922-07-01,NaT,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,87.0
4,BENE11005,1935-09-01,NaT,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200,74.0


In [12]:
#Lets create a new variable 'WhetherDead' with flag 1 means Dead and 0 means not Dead

train_ben.loc[train_ben.DOD.isna(),'WhetherDead']=0
train_ben.loc[train_ben.DOD.notna(),'WhetherDead']=1
train_ben.loc[:,'WhetherDead'].head(7)


0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    1.0
Name: WhetherDead, dtype: float64

Now Add a column of Admit for number of days for inpatient Data(train_in)

In [13]:
## As patient can be admitted for only for 1 day,we will add 1 to the difference of Discharge Date and Admission Date 

train_in['AdmissionDt'] = pd.to_datetime(train_in['AdmissionDt'] , format = '%Y-%m-%d')
train_in['DischargeDt'] = pd.to_datetime(train_in['DischargeDt'],format = '%Y-%m-%d')
train_in['AdmitForDays'] = ((train_in['DischargeDt'] - train_in['AdmissionDt']).dt.days)+1

In [14]:
train_in

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,AdmitForDays
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,3
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,9
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40469,BENE159167,CLM69886,2009-09-28,2009-10-02,PRV53671,7000,PHY345332,PHY319565,NaN,2009-09-28,2859,1068.0,2009-10-02,812,2851,2762,53550,V0254,V1005,5173,79092,70704,2869,4148,9904.0,NaN,NaN,NaN,NaN,NaN,5
40470,BENE159175,CLM74504,2009-11-03,2009-11-06,PRV54981,4000,PHY342806,PHY365497,NaN,2009-11-03,79902,1068.0,2009-11-06,228,42823,4148,4019,3970,4241,V1042,49122,4254,5990,NaN,8605.0,NaN,NaN,NaN,NaN,NaN,4
40471,BENE159177,CLM76485,2009-11-18,2009-11-22,PRV56588,3000,PHY423220,PHY332752,NaN,2009-11-18,78605,1068.0,2009-11-22,302,4280,3963,5859,4019,40390,4111,71534,71590,V4502,NaN,9390.0,NaN,NaN,NaN,NaN,NaN,5
40472,BENE159177,CLM79949,2009-12-17,2009-12-18,PRV56575,5000,PHY353156,NaN,NaN,2009-12-17,7802,1068.0,2009-12-18,309,7802,5859,496,2809,V1254,42822,4263,0414,41400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [15]:
df_in=train_in

In [16]:
df_in.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,AdmitForDays
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,3
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,9
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,18


## From inpatient data we want the new features to get a dataset at provider level
- for diag_codes (e.g.-ClmDiagnosisCode_1,ClmDiagnosisCode_2 etc.) we aggregate them as their count and Unique counts at provider Level
- for Physicians(e.g.-AttendingPhysician,OperatingPhysician,OtherPhysician) we aggregate them as their Unique count at provider Level
- for InscClaimAmtReimbursed we take their 'sum','mean','min','max','std'(standard Deviation) as new features at Provider level

Then we concat these new feature columns at Provider level and get a new dataframe from original dataset train_in

we did this in a function called inpatient()


In [17]:
def inpatient(df):

#     df_agg_cat = df.groupby('Provider')[['ClaimID','BeneID']].agg(['nunique'])
#     df_agg_cat.columns = ['_'.join(x) for x in df_agg_cat.columns]
#     df_agg_cat = df_agg_cat.reset_index()

    diag_codes=['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7','ClmDiagnosisCode_8','ClmDiagnosisCode_9']
    df_agg_diag = df.groupby(['Provider'])[diag_codes].agg(['count','nunique'])
    df_agg_diag.columns = ['_in_'.join(x) for x in df_agg_diag.columns]
    

    Physicians=['AttendingPhysician','OperatingPhysician','OtherPhysician']
    df_agg_phy = df.groupby(['Provider'])[Physicians].agg(['nunique'])
    df_agg_phy.columns = ['_in_'.join(x) for x in df_agg_phy.columns]
    #df_agg_phy = df_agg_phy.reset_index()
    
    df_agg_insc=df.groupby(['Provider'])[['InscClaimAmtReimbursed']].agg(['sum','mean','min','max','std'])
    df_agg_insc.columns = ['_in_'.join(x) for x in df_agg_insc.columns]
    
    df2 = pd.concat([df_agg_diag,df_agg_phy,df_agg_insc], axis=1)
    df2=df2.reset_index()
    return df2
    

In [18]:
df_in=inpatient(train_in)

In [19]:
df_in.head()

,Provider,ClmDiagnosisCode_1_in_count,ClmDiagnosisCode_1_in_nunique,ClmDiagnosisCode_2_in_count,ClmDiagnosisCode_2_in_nunique,ClmDiagnosisCode_3_in_count,ClmDiagnosisCode_3_in_nunique,ClmDiagnosisCode_4_in_count,ClmDiagnosisCode_4_in_nunique,ClmDiagnosisCode_5_in_count,ClmDiagnosisCode_5_in_nunique,ClmDiagnosisCode_6_in_count,ClmDiagnosisCode_6_in_nunique,ClmDiagnosisCode_7_in_count,ClmDiagnosisCode_7_in_nunique,ClmDiagnosisCode_8_in_count,ClmDiagnosisCode_8_in_nunique,ClmDiagnosisCode_9_in_count,ClmDiagnosisCode_9_in_nunique,AttendingPhysician_in_nunique,OperatingPhysician_in_nunique,OtherPhysician_in_nunique,InscClaimAmtReimbursed_in_sum,InscClaimAmtReimbursed_in_mean,InscClaimAmtReimbursed_in_min,InscClaimAmtReimbursed_in_max,InscClaimAmtReimbursed_in_std
0,PRV51001,5,5,5,5,5,5,5,5,5,5,5,5,3,3,2,2,1,1,4,2,1,97000,19400.000000,3000,42000,18352.111595
1,PRV51003,62,52,62,53,61,48,59,47,58,49,54,44,52,45,47,39,44,37,2,2,0,573000,9241.935484,0,57000,8513.606244
2,PRV51007,3,3,3,3,3,3,3,3,3,3,3,3,2,2,1,1,1,1,2,1,0,19000,6333.333333,3000,10000,3511.884584
3,PRV51008,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,2,2,0,25000,12500.000000,4000,21000,12020.815280
4,PRV51011,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,5000,5000.000000,5000,5000,NaN


In [20]:
df_in.shape


(2092, 27)

Unique count of Provider in train_in dataset was 2092. that is the row number of this new dataset

In [21]:
train_in['Provider'].nunique()

2092

In [22]:
df_in[df_in["Provider"]=='PRV51001']

,Provider,ClmDiagnosisCode_1_in_count,ClmDiagnosisCode_1_in_nunique,ClmDiagnosisCode_2_in_count,ClmDiagnosisCode_2_in_nunique,ClmDiagnosisCode_3_in_count,ClmDiagnosisCode_3_in_nunique,ClmDiagnosisCode_4_in_count,ClmDiagnosisCode_4_in_nunique,ClmDiagnosisCode_5_in_count,ClmDiagnosisCode_5_in_nunique,ClmDiagnosisCode_6_in_count,ClmDiagnosisCode_6_in_nunique,ClmDiagnosisCode_7_in_count,ClmDiagnosisCode_7_in_nunique,ClmDiagnosisCode_8_in_count,ClmDiagnosisCode_8_in_nunique,ClmDiagnosisCode_9_in_count,ClmDiagnosisCode_9_in_nunique,AttendingPhysician_in_nunique,OperatingPhysician_in_nunique,OtherPhysician_in_nunique,InscClaimAmtReimbursed_in_sum,InscClaimAmtReimbursed_in_mean,InscClaimAmtReimbursed_in_min,InscClaimAmtReimbursed_in_max,InscClaimAmtReimbursed_in_std
0,PRV51001,5,5,5,5,5,5,5,5,5,5,5,5,3,3,2,2,1,1,4,2,1,97000,19400.0,3000,42000,18352.111595


In [23]:
#df_out[df_out["Provider"]=='PRV51001']

Now we have got df_in as a new dataframe created from Inpatient Data

## From Outpatient Data we create the new features against each provider the same way as we made from Inpatient data

In [24]:
# #Counting the claim duration
# ## As patient can be admitted for only for 1 day,we will add 1 to the difference of Discharge Date and Admission Date 
# train_out['ClaimStartDt'] = pd.to_datetime(train_out['ClaimStartDt'] , format = '%Y-%m-%d')
# train_out['ClaimEndDt'] = pd.to_datetime(train_out['ClaimEndDt'],format = '%Y-%m-%d')
# train_out['Claim_duration'] = ((train_out['ClaimEndDt'] - train_out['ClaimStartDt']).dt.days)+1

In [25]:
train_out.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,V1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,72887,4280,7197,V4577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [26]:
train_out['Provider'].nunique()

5012

In [27]:
def outpatient(df):

#     df_agg_cat = df.groupby('Provider')[['ClaimID','BeneID']].agg(['nunique'])
#     df_agg_cat.columns = ['_'.join(x) for x in df_agg_cat.columns]
#     df_agg_cat = df_agg_cat.reset_index()

    diag_codes=['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7','ClmDiagnosisCode_8','ClmDiagnosisCode_9']
    df_agg_diag = df.groupby(['Provider'])[diag_codes].agg(['count','nunique'])
    df_agg_diag.columns = ['_out_'.join(x) for x in df_agg_diag.columns]
    

    Physicians=['AttendingPhysician','OperatingPhysician','OtherPhysician']
    df_agg_phy = df.groupby(['Provider'])[Physicians].agg(['nunique'])
    df_agg_phy.columns = ['_out_'.join(x) for x in df_agg_phy.columns]
    #df_agg_phy = df_agg_phy.reset_index()
    
    df_agg_insc=df.groupby(['Provider'])[['InscClaimAmtReimbursed']].agg(['sum','mean','min','max','std'])
    df_agg_insc.columns = ['_out_'.join(x) for x in df_agg_insc.columns]
    
    df2 = pd.concat([df_agg_diag,df_agg_phy,df_agg_insc], axis=1)
    df2=df2.reset_index()
    return df2

In [28]:
df_out=outpatient(train_out)

In [29]:
df_out.shape

(5012, 27)

In [30]:
df_out.head()

,Provider,ClmDiagnosisCode_1_out_count,ClmDiagnosisCode_1_out_nunique,ClmDiagnosisCode_2_out_count,ClmDiagnosisCode_2_out_nunique,ClmDiagnosisCode_3_out_count,ClmDiagnosisCode_3_out_nunique,ClmDiagnosisCode_4_out_count,ClmDiagnosisCode_4_out_nunique,ClmDiagnosisCode_5_out_count,ClmDiagnosisCode_5_out_nunique,ClmDiagnosisCode_6_out_count,ClmDiagnosisCode_6_out_nunique,ClmDiagnosisCode_7_out_count,ClmDiagnosisCode_7_out_nunique,ClmDiagnosisCode_8_out_count,ClmDiagnosisCode_8_out_nunique,ClmDiagnosisCode_9_out_count,ClmDiagnosisCode_9_out_nunique,AttendingPhysician_out_nunique,OperatingPhysician_out_nunique,OtherPhysician_out_nunique,InscClaimAmtReimbursed_out_sum,InscClaimAmtReimbursed_out_mean,InscClaimAmtReimbursed_out_min,InscClaimAmtReimbursed_out_max,InscClaimAmtReimbursed_out_std
0,PRV51001,19,18,11,11,5,5,3,3,3,3,3,3,0,0,0,0,0,0,10,3,7,7640,382.000000,10,1500,422.817617
1,PRV51003,70,66,50,41,31,22,19,15,8,8,4,3,3,3,3,3,1,1,44,5,22,32670,466.714286,0,3300,821.364579
2,PRV51004,143,128,91,68,56,52,34,32,24,21,16,12,9,8,8,7,4,4,38,19,26,52170,350.134228,0,3300,689.963754
3,PRV51005,1149,809,723,327,455,232,287,186,164,113,100,79,68,61,43,40,26,25,6,6,4,280910,241.124464,0,4080,491.556392
4,PRV51007,69,68,43,31,26,26,18,18,11,11,9,9,7,7,5,5,4,4,8,4,7,14710,213.188406,0,3300,447.231657


In [31]:
df_out[df_out['Provider']=='PRV55911']

,Provider,ClmDiagnosisCode_1_out_count,ClmDiagnosisCode_1_out_nunique,ClmDiagnosisCode_2_out_count,ClmDiagnosisCode_2_out_nunique,ClmDiagnosisCode_3_out_count,ClmDiagnosisCode_3_out_nunique,ClmDiagnosisCode_4_out_count,ClmDiagnosisCode_4_out_nunique,ClmDiagnosisCode_5_out_count,ClmDiagnosisCode_5_out_nunique,ClmDiagnosisCode_6_out_count,ClmDiagnosisCode_6_out_nunique,ClmDiagnosisCode_7_out_count,ClmDiagnosisCode_7_out_nunique,ClmDiagnosisCode_8_out_count,ClmDiagnosisCode_8_out_nunique,ClmDiagnosisCode_9_out_count,ClmDiagnosisCode_9_out_nunique,AttendingPhysician_out_nunique,OperatingPhysician_out_nunique,OtherPhysician_out_nunique,InscClaimAmtReimbursed_out_sum,InscClaimAmtReimbursed_out_mean,InscClaimAmtReimbursed_out_min,InscClaimAmtReimbursed_out_max,InscClaimAmtReimbursed_out_std
3649,PRV55911,188,166,109,77,74,57,52,44,32,28,26,26,16,14,8,8,7,7,12,9,6,63440,328.704663,0,3300,680.547329


Now we have got df_out as a new dataframe created from Outpatient Data

## Now we have to work with beneficiary Data and make some features and getting dummies for gender and Race variable and then we will merge the data with inpatient and outpatient data

In [32]:
train_ben.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,WhetherDead
0,BENE11001,1943-01-01,NaT,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,67.0,0.0
1,BENE11002,1936-09-01,NaT,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50,73.0,0.0
2,BENE11003,1936-08-01,NaT,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40,73.0,0.0
3,BENE11004,1922-07-01,NaT,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760,87.0,0.0
4,BENE11005,1935-09-01,NaT,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200,74.0,0.0


- first set the values of 2 to 0 for the categorical variables and for RenalDiseaseIndicator the value of y is set to 1

In [33]:
train_ben.RenalDiseaseIndicator.value_counts()

0    118978
Y     19578
Name: RenalDiseaseIndicator, dtype: int64

In [34]:
train_ben = train_ben.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                           'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2, 
                           'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2, 
                           'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2 }, 0)

train_ben = train_ben.replace({'RenalDiseaseIndicator': 'Y'}, 1)

In [35]:
train_ben.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,WhetherDead
0,BENE11001,1943-01-01,NaT,1,1,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0
1,BENE11002,1936-09-01,NaT,2,1,0,39,280,12,12,0,0,0,0,0,0,0,0,0,0,0,0,0,30,50,73.0,0.0
2,BENE11003,1936-08-01,NaT,1,1,0,52,590,12,12,1,0,0,0,0,0,0,1,0,0,0,0,0,90,40,73.0,0.0
3,BENE11004,1922-07-01,NaT,1,1,0,39,270,12,12,1,1,0,0,0,0,1,1,1,1,0,0,0,1810,760,87.0,0.0
4,BENE11005,1935-09-01,NaT,1,1,0,24,680,12,12,0,0,0,0,1,0,1,0,0,0,0,0,0,1790,1200,74.0,0.0


- Now to get dummies for gender and race

In [36]:
train_ben.Race.dtype

dtype('int64')

In [37]:
## Lets Convert types of gender and race to categorical.

train_ben.Gender=train_ben.Gender.astype('category')
train_ben.Race=train_ben.Race.astype('category')

In [38]:
# Lets create dummies for categorrical columns.

train_ben=pd.get_dummies(train_ben,columns=['Gender','Race'])

In [39]:
#after getting dummies lets see the total counts of gender and races

print(train_ben.Gender_1.value_counts())
print(train_ben.Gender_2.value_counts())
print(train_ben.Race_1.value_counts())
print(train_ben.Race_2.value_counts())
print(train_ben.Race_3.value_counts())
print(train_ben.Race_5.value_counts())

0    79106
1    59450
Name: Gender_1, dtype: int64
1    79106
0    59450
Name: Gender_2, dtype: int64
1    117057
0     21499
Name: Race_1, dtype: int64
0    125018
1     13538
Name: Race_2, dtype: int64
0    133497
1      5059
Name: Race_3, dtype: int64
0    135654
1      2902
Name: Race_5, dtype: int64


## Now We Merge the datasets inpatient, outpatient and beneficiary
- First we merge Inpatient and outpatient datas on common columns as outer join and then inner join it with beneficiary data on beneID

In [40]:
[ x for x in train_in.columns if x not in train_out.columns]

['AdmissionDt', 'DischargeDt', 'DiagnosisGroupCode', 'AdmitForDays']

In [41]:

common_columns = [ x for x in train_out.columns if x in train_in.columns]
print(common_columns)
train_in_out = pd.merge(train_in,train_out, left_on = common_columns, right_on = common_columns,how = 'outer')
train_merged=pd.merge(train_in_out,train_ben,left_on='BeneID',right_on='BeneID',how='inner')

#final.drop(['Unnamed: 0'], axis = 1,inplace=True)


['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider', 'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician', 'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid', 'ClmAdmitDiagnosisCode']


In [42]:
train_in_out.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,AdmitForDays
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,3.0
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,9.0
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,18.0


In [43]:
print(train_in.shape)
print(train_out.shape)
print(train_in_out.shape)
print(train_merged.shape)

(40474, 31)
(517737, 27)
(558211, 31)
(558211, 61)


In [44]:
#train_merged=merging(train_provider)

In [45]:
train_merged.head(5)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,AdmitForDays,DOB,DOD,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,WhetherDead,Gender_1,Gender_2,Race_1,Race_2,Race_3,Race_5
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,3.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,9.0,1914-03-01,NaT,0,1,360,12,12,0,1,1,0,0,1,1,0,0,1,1,5000,1068,250,320,96.0,0.0,0,1,0,1,0,0
4,BENE11011,CLM144521,2009-01-18,2009-01-18,PRV52314,50,PHY379398,NaN,NaN,NaT,78900,0.0,NaT,NaN,78969,78701,V5866,59389,2449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1914-03-01,NaT,0,1,360,12,12,0,1,1,0,0,1,1,0,0,1,1,5000,1068,250,320,96.0,0.0,0,1,0,1,0,0


In [46]:
train_merged.shape

(558211, 61)

In [47]:
train_merged['Provider'].nunique()

5410

In [48]:
# #Counting the claim duration
# ## As patient can be admitted for only for 1 day,we will add 1 to the difference of Discharge Date and Admission Date 
train_merged['ClaimStartDt'] = pd.to_datetime(train_merged['ClaimStartDt'] , format = '%Y-%m-%d')
train_merged['ClaimEndDt'] = pd.to_datetime(train_merged['ClaimEndDt'],format = '%Y-%m-%d')
train_merged['Claim_duration'] = ((train_merged['ClaimEndDt'] - train_merged['ClaimStartDt']).dt.days)+1

In [49]:
train_merged.shape

(558211, 62)

In [50]:
train_merged.head(5)

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,AdmitForDays,DOB,DOD,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,WhetherDead,Gender_1,Gender_2,Race_1,Race_2,Race_3,Race_5,Claim_duration
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0,7
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,3.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0,3
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0,4
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,9.0,1914-03-01,NaT,0,1,360,12,12,0,1,1,0,0,1,1,0,0,1,1,5000,1068,250,320,96.0,0.0,0,1,0,1,0,0,9
4,BENE11011,CLM144521,2009-01-18,2009-01-18,PRV52314,50,PHY379398,NaN,NaN,NaT,78900,0.0,NaT,NaN,78969,78701,V5866,59389,2449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1914-03-01,NaT,0,1,360,12,12,0,1,1,0,0,1,1,0,0,1,1,5000,1068,250,320,96.0,0.0,0,1,0,1,0,0,1


Here we get the train_merged data from inpatient, outpatient and beneficiary datasets

## Now create the suspicious flag from the merged data from the provider count and bene_Id count

In [51]:
bene_test =train_merged.groupby('BeneID')[['Provider']].agg(['nunique','count'])
bene_test.columns = ['_'.join(x) for x in bene_test.columns]
bene_test=bene_test.reset_index()

In [52]:
bene_test['Flag_suspicious']=bene_test['Provider_nunique']>1

In [53]:
bene_test.Flag_suspicious.value_counts()

True     91653
False    46903
Name: Flag_suspicious, dtype: int64

In [54]:
bene_test.head()

,BeneID,Provider_nunique,Provider_count,Flag_suspicious
0,BENE100000,1,2,False
1,BENE100001,6,10,True
2,BENE100002,11,14,True
3,BENE100003,1,1,False
4,BENE100004,4,6,True


In [55]:
bene_test.drop(["Provider_nunique",'Provider_count'], axis = 1,inplace=True)

In [56]:
bene_test.dtypes

BeneID             object
Flag_suspicious      bool
dtype: object

In [57]:
#first set datatype as string

bene_test.Flag_suspicious=bene_test.Flag_suspicious.astype('str')



In [58]:
# Now we set the valuse of true as 1 and false as 0

bene_test.Flag_suspicious.replace(['True','False'],['1','0'],inplace=True)
bene_test.Flag_suspicious=bene_test.Flag_suspicious.astype('int')

In [59]:
bene_test.head()

,BeneID,Flag_suspicious
0,BENE100000,0
1,BENE100001,1
2,BENE100002,1
3,BENE100003,0
4,BENE100004,1


## now merge this bene_test data with the merged data to get the suspicious flag into the data at bene_id level

In [60]:
train_merged_with_flag=pd.merge(train_merged,bene_test,on='BeneID', how='left')

In [61]:
train_merged_with_flag.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,AdmitForDays,DOB,DOD,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,WhetherDead,Gender_1,Gender_2,Race_1,Race_2,Race_3,Race_5,Claim_duration,Flag_suspicious
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0,7,1
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,3.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0,3,1
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1943-01-01,NaT,0,39,230,12,12,1,0,1,0,0,1,1,1,0,1,1,36000,3204,60,70,67.0,0.0,1,0,1,0,0,0,4,1
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,9.0,1914-03-01,NaT,0,1,360,12,12,0,1,1,0,0,1,1,0,0,1,1,5000,1068,250,320,96.0,0.0,0,1,0,1,0,0,9,1
4,BENE11011,CLM144521,2009-01-18,2009-01-18,PRV52314,50,PHY379398,NaN,NaN,NaT,78900,0.0,NaT,NaN,78969,78701,V5866,59389,2449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1914-03-01,NaT,0,1,360,12,12,0,1,1,0,0,1,1,0,0,1,1,5000,1068,250,320,96.0,0.0,0,1,0,1,0,0,1,1


In [62]:
train_merged_with_flag.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558211 entries, 0 to 558210
Data columns (total 63 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   BeneID                           558211 non-null  object        
 1   ClaimID                          558211 non-null  object        
 2   ClaimStartDt                     558211 non-null  datetime64[ns]
 3   ClaimEndDt                       558211 non-null  datetime64[ns]
 4   Provider                         558211 non-null  object        
 5   InscClaimAmtReimbursed           558211 non-null  int64         
 6   AttendingPhysician               556703 non-null  object        
 7   OperatingPhysician               114447 non-null  object        
 8   OtherPhysician                   199736 non-null  object        
 9   AdmissionDt                      40474 non-null   datetime64[ns]
 10  ClmAdmitDiagnosisCode            145899 non-

## Now to get features from Merged dataset

- Here we created various new features aggregated by unique counts, counts , mean, min, max, std deviation etc. at provider level
- for the flag suspicious it tells if the beneficiary is suspicious or not(1 or 0). Now we aggregate them by taking their sum at each provider level. It tells the count of suspicious beneficiaries under a provider.
- similarly we take counts of genders ,races, chronic diseases at each provider level

Finally we merge them to get a new dataframe df_merged

In [63]:
def merged(df):
    df_agg_cat = df.groupby('Provider')[['ClaimID','BeneID']].agg(['nunique','count'])
    df_agg_cat.columns = ['_'.join(x) for x in df_agg_cat.columns]
    
    df_agg_insc=df.groupby(['Provider'])[['InscClaimAmtReimbursed']].agg(['sum','mean','min','max','std'])
    df_agg_insc.columns = ['_merged_'.join(x) for x in df_agg_insc.columns]
    
    
    reimbursement=['IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt']
    df_agg_reimbursement=df.groupby(['Provider'])[reimbursement].agg(['mean','min','max','std'])
    df_agg_reimbursement.columns = ['_merged_'.join(x) for x in df_agg_reimbursement.columns]
    
    df_agg_clmdur=df.groupby(['Provider'])[['Claim_duration']].agg(['mean'])
    df_agg_clmdur.columns = ['_merged_'.join(x) for x in df_agg_clmdur.columns]
    
    gen_race_diseases=['Gender_1','Gender_2','Race_1','Race_2','Race_3','Race_5','ChronicCond_Alzheimer','ChronicCond_Heartfailure','ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary','ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis','ChronicCond_rheumatoidarthritis','ChronicCond_stroke']
    df_agg_gen_race_diseases=df.groupby(['Provider'])[gen_race_diseases].agg(['sum'])
    df_agg_gen_race_diseases.columns = ['_merged_'.join(x) for x in df_agg_gen_race_diseases.columns]
    
    df_agg_age=df.groupby(['Provider'])[['Age']].agg(['mean','min','max'])
    df_agg_age.columns = ['_merged_'.join(x) for x in df_agg_age.columns]
    
    df_agg_flag=df.groupby(['Provider'])[['Flag_suspicious']].agg(['sum'])
    df_agg_flag.columns = ['_merged_'.join(x) for x in df_agg_flag.columns]
    
    
    
    df2 = pd.concat([df_agg_cat,df_agg_insc,df_agg_reimbursement,df_agg_gen_race_diseases,df_agg_clmdur,df_agg_age,df_agg_flag], axis=1)
    df2=df2.reset_index()
    return df2
    

In [64]:
df_merged=merged(train_merged_with_flag)

In [65]:
df_merged.shape

(5410, 48)

In [66]:
df_merged.head(15)

,Provider,ClaimID_nunique,ClaimID_count,BeneID_nunique,BeneID_count,InscClaimAmtReimbursed_merged_sum,InscClaimAmtReimbursed_merged_mean,InscClaimAmtReimbursed_merged_min,InscClaimAmtReimbursed_merged_max,InscClaimAmtReimbursed_merged_std,IPAnnualReimbursementAmt_merged_mean,IPAnnualReimbursementAmt_merged_min,IPAnnualReimbursementAmt_merged_max,IPAnnualReimbursementAmt_merged_std,IPAnnualDeductibleAmt_merged_mean,IPAnnualDeductibleAmt_merged_min,IPAnnualDeductibleAmt_merged_max,IPAnnualDeductibleAmt_merged_std,OPAnnualReimbursementAmt_merged_mean,OPAnnualReimbursementAmt_merged_min,OPAnnualReimbursementAmt_merged_max,OPAnnualReimbursementAmt_merged_std,OPAnnualDeductibleAmt_merged_mean,OPAnnualDeductibleAmt_merged_min,OPAnnualDeductibleAmt_merged_max,OPAnnualDeductibleAmt_merged_std,Gender_1_merged_sum,Gender_2_merged_sum,Race_1_merged_sum,Race_2_merged_sum,Race_3_merged_sum,Race_5_merged_sum,ChronicCond_Alzheimer_merged_sum,ChronicCond_Heartfailure_merged_sum,ChronicCond_KidneyDisease_merged_sum,ChronicCond_Cancer_merged_sum,ChronicCond_ObstrPulmonary_merged_sum,ChronicCond_Depression_merged_sum,ChronicCond_Diabetes_merged_sum,ChronicCond_IschemicHeart_merged_sum,ChronicCond_Osteoporasis_merged_sum,ChronicCond_rheumatoidarthritis_merged_sum,ChronicCond_stroke_merged_sum,Claim_duration_merged_mean,Age_merged_mean,Age_merged_min,Age_merged_max,Flag_suspicious_merged_sum
0,PRV51001,25,25,24,25,104640,4185.600000,10,42000,10796.091144,17606.000000,0,143800,38895.477458,897.120000,0,4272,1332.505670,2615.200000,0,13460,2974.176525,463.920000,0,2700,635.782321,9.0,16.0,21.0,4.0,0.0,0.0,15,19,17,5,10,9,21,23,6,8,6,2.440000,78.840000,57.0,98.0,24
1,PRV51003,132,132,117,132,605670,4588.409091,0,57000,7309.794729,7568.181818,0,57000,10040.975578,931.424242,0,7208,1059.574926,2678.181818,0,31880,5190.714819,737.121212,0,9950,1517.084151,54.0,78.0,107.0,24.0,0.0,1.0,56,80,64,10,41,54,100,112,33,38,12,4.674242,70.022727,38.0,97.0,117
2,PRV51004,149,149,138,149,52170,350.134228,0,3300,689.963754,4351.879195,0,51500,9889.562978,434.953020,0,7136,945.236867,2194.899329,0,20900,3198.010914,622.751678,0,7410,873.191458,46.0,103.0,120.0,24.0,5.0,0.0,64,88,50,16,41,63,105,108,49,46,17,2.429530,72.161074,26.0,100.0,136
3,PRV51005,1165,1165,495,1165,280910,241.124464,0,4080,491.556392,3623.991416,0,144000,9688.526566,379.162232,0,7136,751.597235,2109.733906,10,97510,3960.211085,636.328755,0,6330,845.696041,511.0,654.0,893.0,262.0,10.0,0.0,426,680,507,165,295,485,799,895,344,331,124,2.088412,70.475536,29.0,101.0,940
4,PRV51007,72,72,58,72,33710,468.194444,0,10000,1433.769116,3050.000000,0,24600,5682.560175,445.000000,0,3204,796.600594,1729.722222,30,6060,1623.875535,469.722222,0,1970,518.885513,34.0,38.0,58.0,14.0,0.0,0.0,26,40,22,12,16,29,49,51,21,22,12,1.958333,69.291667,26.0,99.0,66
5,PRV51008,43,43,36,43,35630,828.604651,0,21000,3224.765463,6592.093023,0,90000,15111.833988,620.930233,0,4272,995.142808,2570.465116,50,20500,3546.637545,616.046512,0,4140,741.067889,19.0,24.0,34.0,9.0,0.0,0.0,17,25,14,9,9,12,30,33,11,6,2,2.534884,75.627907,29.0,99.0,37
6,PRV51011,58,58,53,58,16630,286.724138,0,5000,701.047807,4414.827586,0,32000,7314.270656,674.275862,0,8136,1280.311390,1399.310345,60,6340,1326.395805,366.724138,10,1070,275.753436,27.0,31.0,51.0,7.0,0.0,0.0,25,37,25,4,21,31,41,44,20,16,2,1.551724,74.379310,37.0,100.0,55
7,PRV51012,48,48,31,48,16890,351.875000,0,3300,707.893658,6935.416667,0,30700,10616.797000,839.916667,0,3204,1235.219471,3793.958333,30,33850,8052.692094,735.208333,0,5290,1309.427339,11.0,37.0,41.0,7.0,0.0,0.0,18,26,22,6,21,19,35,38,17,17,3,4.708333,74.479167,43.0,97.0,36
8,PRV51013,46,46,44,46,14880,323.478261,0,1800,471.622341,3635.652174,0,36200,7643.826747,593.304348,0,10204,1616.480249,3226.521739,0,31880,5385.073699,914.782609,0,9950,1523.187796,12.0,34.0,43.0,3.0,0.0,0.0,17,26,19,4,16,18,37,36,12,16,2,2.456522,74.391304,28.0,101.0,45
9,PRV51014,30,30,25,30,10480,349.333333,0,3300,822.984197,2786.333333,0,13000,4484.440905,4

In [67]:
df_merged[df_merged['Provider'] == 'PRV55911']

,Provider,ClaimID_nunique,ClaimID_count,BeneID_nunique,BeneID_count,InscClaimAmtReimbursed_merged_sum,InscClaimAmtReimbursed_merged_mean,InscClaimAmtReimbursed_merged_min,InscClaimAmtReimbursed_merged_max,InscClaimAmtReimbursed_merged_std,IPAnnualReimbursementAmt_merged_mean,IPAnnualReimbursementAmt_merged_min,IPAnnualReimbursementAmt_merged_max,IPAnnualReimbursementAmt_merged_std,IPAnnualDeductibleAmt_merged_mean,IPAnnualDeductibleAmt_merged_min,IPAnnualDeductibleAmt_merged_max,IPAnnualDeductibleAmt_merged_std,OPAnnualReimbursementAmt_merged_mean,OPAnnualReimbursementAmt_merged_min,OPAnnualReimbursementAmt_merged_max,OPAnnualReimbursementAmt_merged_std,OPAnnualDeductibleAmt_merged_mean,OPAnnualDeductibleAmt_merged_min,OPAnnualDeductibleAmt_merged_max,OPAnnualDeductibleAmt_merged_std,Gender_1_merged_sum,Gender_2_merged_sum,Race_1_merged_sum,Race_2_merged_sum,Race_3_merged_sum,Race_5_merged_sum,ChronicCond_Alzheimer_merged_sum,ChronicCond_Heartfailure_merged_sum,ChronicCond_KidneyDisease_merged_sum,ChronicCond_Cancer_merged_sum,ChronicCond_ObstrPulmonary_merged_sum,ChronicCond_Depression_merged_sum,ChronicCond_Diabetes_merged_sum,ChronicCond_IschemicHeart_merged_sum,ChronicCond_Osteoporasis_merged_sum,ChronicCond_rheumatoidarthritis_merged_sum,ChronicCond_stroke_merged_sum,Claim_duration_merged_mean,Age_merged_mean,Age_merged_min,Age_merged_max,Flag_suspicious_merged_sum
3927,PRV55911,239,239,136,239,465440,1947.447699,0,30000,4341.202222,5778.033473,0,51010,10091.554081,688.167364,0,5340,1037.89119,1727.447699,0,15140,2379.585571,492.635983,0,4430,736.265387,109.0,130.0,213.0,22.0,1.0,3.0,90,153,105,39,70,110,161,190,84,58,28,3.903766,71.933054,27.0,97.0,214


# Now we have three Datasets ready-- df_in,df_out,df_merged
## Now we have to merge them and the whole data must be merged with train_Provider

Now merge these 3 dataframes on provider level by left join and i need to replace the NAN value by 0's
: Then save it as a csv file

In [68]:
df= pd.merge(df_merged,df_out,on='Provider', how='left')

In [69]:
df.head(2)

,Provider,ClaimID_nunique,ClaimID_count,BeneID_nunique,BeneID_count,InscClaimAmtReimbursed_merged_sum,InscClaimAmtReimbursed_merged_mean,InscClaimAmtReimbursed_merged_min,InscClaimAmtReimbursed_merged_max,InscClaimAmtReimbursed_merged_std,IPAnnualReimbursementAmt_merged_mean,IPAnnualReimbursementAmt_merged_min,IPAnnualReimbursementAmt_merged_max,IPAnnualReimbursementAmt_merged_std,IPAnnualDeductibleAmt_merged_mean,IPAnnualDeductibleAmt_merged_min,IPAnnualDeductibleAmt_merged_max,IPAnnualDeductibleAmt_merged_std,OPAnnualReimbursementAmt_merged_mean,OPAnnualReimbursementAmt_merged_min,OPAnnualReimbursementAmt_merged_max,OPAnnualReimbursementAmt_merged_std,OPAnnualDeductibleAmt_merged_mean,OPAnnualDeductibleAmt_merged_min,OPAnnualDeductibleAmt_merged_max,OPAnnualDeductibleAmt_merged_std,Gender_1_merged_sum,Gender_2_merged_sum,Race_1_merged_sum,Race_2_merged_sum,Race_3_merged_sum,Race_5_merged_sum,ChronicCond_Alzheimer_merged_sum,ChronicCond_Heartfailure_merged_sum,ChronicCond_KidneyDisease_merged_sum,ChronicCond_Cancer_merged_sum,ChronicCond_ObstrPulmonary_merged_sum,ChronicCond_Depression_merged_sum,ChronicCond_Diabetes_merged_sum,ChronicCond_IschemicHeart_merged_sum,ChronicCond_Osteoporasis_merged_sum,ChronicCond_rheumatoidarthritis_merged_sum,ChronicCond_stroke_merged_sum,Claim_duration_merged_mean,Age_merged_mean,Age_merged_min,Age_merged_max,Flag_suspicious_merged_sum,ClmDiagnosisCode_1_out_count,ClmDiagnosisCode_1_out_nunique,ClmDiagnosisCode_2_out_count,ClmDiagnosisCode_2_out_nunique,ClmDiagnosisCode_3_out_count,ClmDiagnosisCode_3_out_nunique,ClmDiagnosisCode_4_out_count,ClmDiagnosisCode_4_out_nunique,ClmDiagnosisCode_5_out_count,ClmDiagnosisCode_5_out_nunique,ClmDiagnosisCode_6_out_count,ClmDiagnosisCode_6_out_nunique,ClmDiagnosisCode_7_out_count,ClmDiagnosisCode_7_out_nunique,ClmDiagnosisCode_8_out_count,ClmDiagnosisCode_8_out_nunique,ClmDiagnosisCode_9_out_count,ClmDiagnosisCode_9_out_nunique,AttendingPhysician_out_nunique,OperatingPhysician_out_nunique,OtherPhysician_out_nunique,InscClaimAmtReimbursed_out_sum,InscClaimAmtReimbursed_out_mean,InscClaimAmtReimbursed_out_min,InscClaimAmtReimbursed_out_max,InscClaimAmtReimbursed_out_std
0,PRV51001,25,25,24,25,104640,4185.600000,10,42000,10796.091144,17606.000000,0,143800,38895.477458,897.120000,0,4272,1332.505670,2615.200000,0,13460,2974.176525,463.920000,0,2700,635.782321,9.0,16.0,21.0,4.0,0.0,0.0,15,19,17,5,10,9,21,23,6,8,6,2.440000,78.840000,57.0,98.0,24,19.0,18.0,11.0,11.0,5.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,3.0,7.0,7640.0,382.000000,10.0,1500.0,422.817617
1,PRV51003,132,132,117,132,605670,4588.409091,0,57000,7309.794729,7568.181818,0,57000,10040.975578,931.424242,0,7208,1059.574926,2678.181818,0,31880,5190.714819,737.121212,0,9950,1517.084151,54.0,78.0,107.0,24.0,0.0,1.0,56,80,64,10,41,54,100,112,33,38,12,4.674242,70.022727,38.0,97.0,117,70.0,66.0,50.0,41.0,31.0,22.0,19.0,15.0,8.0,8.0,4.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,44.0,5.0,22.0,32670.0,466.714286,0.0,3300.0,821.364579


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5410 entries, 0 to 5409
Data columns (total 74 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Provider                                    5410 non-null   object 
 1   ClaimID_nunique                             5410 non-null   int64  
 2   ClaimID_count                               5410 non-null   int64  
 3   BeneID_nunique                              5410 non-null   int64  
 4   BeneID_count                                5410 non-null   int64  
 5   InscClaimAmtReimbursed_merged_sum           5410 non-null   int64  
 6   InscClaimAmtReimbursed_merged_mean          5410 non-null   float64
 7   InscClaimAmtReimbursed_merged_min           5410 non-null   int64  
 8   InscClaimAmtReimbursed_merged_max           5410 non-null   int64  
 9   InscClaimAmtReimbursed_merged_std           5170 non-null   float64
 10  IPAnnualReim

In [71]:
df= pd.merge(df,df_in,on='Provider', how='left')

In [72]:
df.head(26)

,Provider,ClaimID_nunique,ClaimID_count,BeneID_nunique,BeneID_count,InscClaimAmtReimbursed_merged_sum,InscClaimAmtReimbursed_merged_mean,InscClaimAmtReimbursed_merged_min,InscClaimAmtReimbursed_merged_max,InscClaimAmtReimbursed_merged_std,IPAnnualReimbursementAmt_merged_mean,IPAnnualReimbursementAmt_merged_min,IPAnnualReimbursementAmt_merged_max,IPAnnualReimbursementAmt_merged_std,IPAnnualDeductibleAmt_merged_mean,IPAnnualDeductibleAmt_merged_min,IPAnnualDeductibleAmt_merged_max,IPAnnualDeductibleAmt_merged_std,OPAnnualReimbursementAmt_merged_mean,OPAnnualReimbursementAmt_merged_min,OPAnnualReimbursementAmt_merged_max,OPAnnualReimbursementAmt_merged_std,OPAnnualDeductibleAmt_merged_mean,OPAnnualDeductibleAmt_merged_min,OPAnnualDeductibleAmt_merged_max,OPAnnualDeductibleAmt_merged_std,Gender_1_merged_sum,Gender_2_merged_sum,Race_1_merged_sum,Race_2_merged_sum,Race_3_merged_sum,Race_5_merged_sum,ChronicCond_Alzheimer_merged_sum,ChronicCond_Heartfailure_merged_sum,ChronicCond_KidneyDisease_merged_sum,ChronicCond_Cancer_merged_sum,ChronicCond_ObstrPulmonary_merged_sum,ChronicCond_Depression_merged_sum,ChronicCond_Diabetes_merged_sum,ChronicCond_IschemicHeart_merged_sum,ChronicCond_Osteoporasis_merged_sum,ChronicCond_rheumatoidarthritis_merged_sum,ChronicCond_stroke_merged_sum,Claim_duration_merged_mean,Age_merged_mean,Age_merged_min,Age_merged_max,Flag_suspicious_merged_sum,ClmDiagnosisCode_1_out_count,ClmDiagnosisCode_1_out_nunique,ClmDiagnosisCode_2_out_count,ClmDiagnosisCode_2_out_nunique,ClmDiagnosisCode_3_out_count,ClmDiagnosisCode_3_out_nunique,ClmDiagnosisCode_4_out_count,ClmDiagnosisCode_4_out_nunique,ClmDiagnosisCode_5_out_count,ClmDiagnosisCode_5_out_nunique,ClmDiagnosisCode_6_out_count,ClmDiagnosisCode_6_out_nunique,ClmDiagnosisCode_7_out_count,ClmDiagnosisCode_7_out_nunique,ClmDiagnosisCode_8_out_count,ClmDiagnosisCode_8_out_nunique,ClmDiagnosisCode_9_out_count,ClmDiagnosisCode_9_out_nunique,AttendingPhysician_out_nunique,OperatingPhysician_out_nunique,OtherPhysician_out_nunique,InscClaimAmtReimbursed_out_sum,InscClaimAmtReimbursed_out_mean,InscClaimAmtReimbursed_out_min,InscClaimAmtReimbursed_out_max,InscClaimAmtReimbursed_out_std,ClmDiagnosisCode_1_in_count,ClmDiagnosisCode_1_in_nunique,ClmDiagnosisCode_2_in_count,ClmDiagnosisCode_2_in_nunique,ClmDiagnosisCode_3_in_count,ClmDiagnosisCode_3_in_nunique,ClmDiagnosisCode_4_in_count,ClmDiagnosisCode_4_in_nunique,ClmDiagnosisCode_5_in_count,ClmDiagnosisCode_5_in_nunique,ClmDiagnosisCode_6_in_count,ClmDiagnosisCode_6_in_nunique,ClmDiagnosisCode_7_in_count,ClmDiagnosisCode_7_in_nunique,ClmDiagnosisCode_8_in_count,ClmDiagnosisCode_8_in_nunique,ClmDiagnosisCode_9_in_count,ClmDiagnosisCode_9_in_nunique,AttendingPhysician_in_nunique,OperatingPhysician_in_nunique,OtherPhysician_in_nunique,InscClaimAmtReimbursed_in_sum,InscClaimAmtReimbursed_in_mean,InscClaimAmtReimbursed_in_min,InscClaimAmtReimbursed_in_max,InscClaimAmtReimbursed_in_std
0,PRV51001,25,25,24,25,104640,4185.600000,10,42000,10796.091144,17606.000000,0,143800,38895.477458,897.120000,0,4272,1332.505670,2615.200000,0,13460,2974.176525,463.920000,0,2700,635.782321,9.0,16.0,21.0,4.0,0.0,0.0,15,19,17,5,10,9,21,23,6,8,6,2.440000,78.840000,57.0,98.0,24,19.0,18.0,11.0,11.0,5.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,3.0,7.0,7640.0,382.000000,10.0,1500.0,422.817617,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,2.0,2.0,1.0,1.0,4.0,2.0,1.0,97000.0,19400.000000,3000.0,42000.0,18352.111595
1,PRV51003,132,132,117,132,605670,4588.409091,0,57000,7309.794729,7568.181818,0,57000,10040.975578,931.424242,0,7208,1059.574926,2678.181818,0,31880,5190.714819,737.121212,0,9950,1517.084151,54.0,78.0,107.0,24.0,0.0,1.0,56,80,64,10,41,54,100,112,33,38,12,4.674242,70.022727,38.0,97.0,117,70.0,66.0,50.0,41.0,31.0,22.0,19.0,15.0,8.0,8.0,4.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,44.0,5.0,22.0,32670.0,466.714286,0.0,3300.0,821.364579,62.0,52.0,62.0,53.0,61.0,48.0,59.0,47.0,58.0,49.0,54.0,44.0,52.0,45.0,47.0,39.0,44.0,37.0,2.0,2.0,0.0

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5410 entries, 0 to 5409
Data columns (total 100 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Provider                                    5410 non-null   object 
 1   ClaimID_nunique                             5410 non-null   int64  
 2   ClaimID_count                               5410 non-null   int64  
 3   BeneID_nunique                              5410 non-null   int64  
 4   BeneID_count                                5410 non-null   int64  
 5   InscClaimAmtReimbursed_merged_sum           5410 non-null   int64  
 6   InscClaimAmtReimbursed_merged_mean          5410 non-null   float64
 7   InscClaimAmtReimbursed_merged_min           5410 non-null   int64  
 8   InscClaimAmtReimbursed_merged_max           5410 non-null   int64  
 9   InscClaimAmtReimbursed_merged_std           5170 non-null   float64
 10  IPAnnualRei

In [74]:
print(df.isnull().sum())

Provider                             0
ClaimID_nunique                      0
ClaimID_count                        0
BeneID_nunique                       0
BeneID_count                         0
                                  ... 
InscClaimAmtReimbursed_in_sum     3318
InscClaimAmtReimbursed_in_mean    3318
InscClaimAmtReimbursed_in_min     3318
InscClaimAmtReimbursed_in_max     3318
InscClaimAmtReimbursed_in_std     3533
Length: 100, dtype: int64


- As we can see that for some columns there are various missing values..
we impute missing values for them with 0 as it is somewhat logical that as for InscClaimAmtReimbursed_in_sum if there is any missing value for a provider then it is assumed that there is not any data for that provider in Inpatient data....if we impute that with mean or median or any other value then it will be illogical...so missing value impute with 0 is the closest logical assumption

In [75]:
df=df.fillna(0)

In [76]:
print(df.isnull().sum())

Provider                          0
ClaimID_nunique                   0
ClaimID_count                     0
BeneID_nunique                    0
BeneID_count                      0
                                 ..
InscClaimAmtReimbursed_in_sum     0
InscClaimAmtReimbursed_in_mean    0
InscClaimAmtReimbursed_in_min     0
InscClaimAmtReimbursed_in_max     0
InscClaimAmtReimbursed_in_std     0
Length: 100, dtype: int64


In [77]:
df.to_csv("C:\\Users\\Sahil\\Desktop\\Summer\\Project\\df_healthcare.csv", index=False)

In [78]:
health=pd.read_csv("C:\\Users\\Sahil\\Desktop\\Summer\\Project\\df_healthcare.csv")

In [113]:
health.head(5)

,Provider,ClaimID_nunique,ClaimID_count,BeneID_nunique,BeneID_count,InscClaimAmtReimbursed_merged_sum,InscClaimAmtReimbursed_merged_mean,InscClaimAmtReimbursed_merged_min,InscClaimAmtReimbursed_merged_max,InscClaimAmtReimbursed_merged_std,IPAnnualReimbursementAmt_merged_mean,IPAnnualReimbursementAmt_merged_min,IPAnnualReimbursementAmt_merged_max,IPAnnualReimbursementAmt_merged_std,IPAnnualDeductibleAmt_merged_mean,IPAnnualDeductibleAmt_merged_min,IPAnnualDeductibleAmt_merged_max,IPAnnualDeductibleAmt_merged_std,OPAnnualReimbursementAmt_merged_mean,OPAnnualReimbursementAmt_merged_min,OPAnnualReimbursementAmt_merged_max,OPAnnualReimbursementAmt_merged_std,OPAnnualDeductibleAmt_merged_mean,OPAnnualDeductibleAmt_merged_min,OPAnnualDeductibleAmt_merged_max,OPAnnualDeductibleAmt_merged_std,Gender_1_merged_sum,Gender_2_merged_sum,Race_1_merged_sum,Race_2_merged_sum,Race_3_merged_sum,Race_5_merged_sum,ChronicCond_Alzheimer_merged_sum,ChronicCond_Heartfailure_merged_sum,ChronicCond_KidneyDisease_merged_sum,ChronicCond_Cancer_merged_sum,ChronicCond_ObstrPulmonary_merged_sum,ChronicCond_Depression_merged_sum,ChronicCond_Diabetes_merged_sum,ChronicCond_IschemicHeart_merged_sum,ChronicCond_Osteoporasis_merged_sum,ChronicCond_rheumatoidarthritis_merged_sum,ChronicCond_stroke_merged_sum,Claim_duration_merged_mean,Age_merged_mean,Age_merged_min,Age_merged_max,Flag_suspicious_merged_sum,ClmDiagnosisCode_1_out_count,ClmDiagnosisCode_1_out_nunique,ClmDiagnosisCode_2_out_count,ClmDiagnosisCode_2_out_nunique,ClmDiagnosisCode_3_out_count,ClmDiagnosisCode_3_out_nunique,ClmDiagnosisCode_4_out_count,ClmDiagnosisCode_4_out_nunique,ClmDiagnosisCode_5_out_count,ClmDiagnosisCode_5_out_nunique,ClmDiagnosisCode_6_out_count,ClmDiagnosisCode_6_out_nunique,ClmDiagnosisCode_7_out_count,ClmDiagnosisCode_7_out_nunique,ClmDiagnosisCode_8_out_count,ClmDiagnosisCode_8_out_nunique,ClmDiagnosisCode_9_out_count,ClmDiagnosisCode_9_out_nunique,AttendingPhysician_out_nunique,OperatingPhysician_out_nunique,OtherPhysician_out_nunique,InscClaimAmtReimbursed_out_sum,InscClaimAmtReimbursed_out_mean,InscClaimAmtReimbursed_out_min,InscClaimAmtReimbursed_out_max,InscClaimAmtReimbursed_out_std,ClmDiagnosisCode_1_in_count,ClmDiagnosisCode_1_in_nunique,ClmDiagnosisCode_2_in_count,ClmDiagnosisCode_2_in_nunique,ClmDiagnosisCode_3_in_count,ClmDiagnosisCode_3_in_nunique,ClmDiagnosisCode_4_in_count,ClmDiagnosisCode_4_in_nunique,ClmDiagnosisCode_5_in_count,ClmDiagnosisCode_5_in_nunique,ClmDiagnosisCode_6_in_count,ClmDiagnosisCode_6_in_nunique,ClmDiagnosisCode_7_in_count,ClmDiagnosisCode_7_in_nunique,ClmDiagnosisCode_8_in_count,ClmDiagnosisCode_8_in_nunique,ClmDiagnosisCode_9_in_count,ClmDiagnosisCode_9_in_nunique,AttendingPhysician_in_nunique,OperatingPhysician_in_nunique,OtherPhysician_in_nunique,InscClaimAmtReimbursed_in_sum,InscClaimAmtReimbursed_in_mean,InscClaimAmtReimbursed_in_min,InscClaimAmtReimbursed_in_max,InscClaimAmtReimbursed_in_std
0,PRV51001,25,25,24,25,104640,4185.600000,10,42000,10796.091144,17606.000000,0,143800,38895.477458,897.120000,0,4272,1332.505670,2615.200000,0,13460,2974.176525,463.920000,0,2700,635.782321,9.0,16.0,21.0,4.0,0.0,0.0,15,19,17,5,10,9,21,23,6,8,6,2.440000,78.840000,57.0,98.0,24,19.0,18.0,11.0,11.0,5.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,3.0,7.0,7640.0,382.000000,10.0,1500.0,422.817617,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,3.0,2.0,2.0,1.0,1.0,4.0,2.0,1.0,97000.0,19400.000000,3000.0,42000.0,18352.111595
1,PRV51003,132,132,117,132,605670,4588.409091,0,57000,7309.794729,7568.181818,0,57000,10040.975578,931.424242,0,7208,1059.574926,2678.181818,0,31880,5190.714819,737.121212,0,9950,1517.084151,54.0,78.0,107.0,24.0,0.0,1.0,56,80,64,10,41,54,100,112,33,38,12,4.674242,70.022727,38.0,97.0,117,70.0,66.0,50.0,41.0,31.0,22.0,19.0,15.0,8.0,8.0,4.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,44.0,5.0,22.0,32670.0,466.714286,0.0,3300.0,821.364579,62.0,52.0,62.0,53.0,61.0,48.0,59.0,47.0,58.0,49.0,54.0,44.0,52.0,45.0,47.0,39.0,44.0,37.0,2.0,2.0,0.0

In [112]:
health.shape

(5410, 100)

In [80]:
#df=train_merged_with_flag[train_merged_with_flag['Provider']== 'PRV55911']

In [81]:
#df['Flag_suspicious'].sum()

In [82]:
#df['Flag_suspicious'].value_counts()

In [83]:
#df['Age'].min()

In [84]:
#df['ChronicCond_KidneyDisease'].value_counts()

In [85]:
#df = train_in[train_in['Provider'] == 'PRV55911']

In [86]:
# df['AdmissionDt'] = pd.to_datetime(df['AdmissionDt'] , format = '%Y-%m-%d')
# df['DischargeDt'] = pd.to_datetime(df['DischargeDt'],format = '%Y-%m-%d')
# df['AdmitForDays'] = ((df['DischargeDt'] - df['AdmissionDt']).dt.days)+1

In [87]:
#df.shape

In [88]:
#pd.set_option('display.max_columns', None)

In [89]:
#df.head()

In [90]:
# df_agg_cat = df.groupby('Provider')[['ClaimID','BeneID']].agg(['nunique'])
# df_agg_cat.columns = ['_'.join(x) for x in df_agg_cat.columns]
# df_agg_cat = df_agg_cat.reset_index()

In [91]:
#df_agg_cat

In [92]:
# diag_codes=['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7','ClmDiagnosisCode_8','ClmDiagnosisCode_9']
# df_agg_diag = df.groupby(['Provider'])[diag_codes].agg(['count','nunique'])
# df_agg_diag.columns = ['_'.join(x) for x in df_agg_diag.columns]
# df_agg_diag = df_agg_diag.reset_index()

In [93]:
# df_agg_diag

In [94]:
# Physicians=['AttendingPhysician','OperatingPhysician','OtherPhysician']
# df_agg_phy = df.groupby(['Provider'])[Physicians].agg(['nunique'])
# df_agg_phy.columns = ['_'.join(x) for x in df_agg_phy.columns]
# df_agg_phy = df_agg_phy.reset_index()

In [95]:
#df_agg_phy

In [96]:
# Physicians=['AttendingPhysician','OperatingPhysician','OtherPhysician']
# df_agg_phy = df.groupby(['Provider'])[Physicians].agg(['count'])
# df_agg_phy.columns = ['_'.join(x) for x in df_agg_phy.columns]
# df_agg_phy = df_agg_phy.reset_index()

In [97]:
#df_agg_phy

In [98]:
#df["ClmDiagnosisCode_8"].count()

In [99]:
#df.ClmDiagnosisCode_8

In [100]:
#df['AttendingPhysician'].count()

In [101]:
#in outpatient data

In [102]:
#df = train_out[train_out['Provider'] == 'PRV55911']

In [103]:
#df.shape

In [104]:
#df.head()

In [105]:
#df['BeneID'].nunique()
#df['BeneID'].count()

In [106]:
# df['InscClaimAmtReimbursed'].mean()

In [107]:
#df['InscClaimAmtReimbursed'].max()

In [108]:
#df['InscClaimAmtReimbursed'].std()

In [109]:
#df = train_merged[train_merged['Provider'] == 'PRV55911']

In [110]:
#df.shape

In [111]:
#df["OPAnnualDeductibleAmt"].mean()